# SM Prototype 10K Train/Test Split, Vectorization, Linear Regression

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
from utils import clean_text
import edgar
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

# Expand the max width of how our dataFrames display on screen
pd.options.display.max_colwidth = 500

C:\Users\yural\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yural\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### not needed for now, only has docs
def ten_k_files():
    company_dict = {"FANG": "0001539838", "RSPP":"0001588216", "CXO":" 0001358071", "MDU": "0000067716",
               "LPI": "0001528129", "NFX": "0000912750", "ECA": "0001157806", "GPOR": "0000874499",
               "RRC": "0000315852", "EQT": "0000033213", "SM": "0000893538"}
    docs_df = pd.DataFrame(columns=['Filing_Type','Filing_Date','Company','Text','Label'])
    for comp in company_dict:
        year = 2019
        cik = company_dict[comp]
        company = edgar.Company(comp, cik)
        tree = company.getAllFilings(filingType = "10-K")
        docs = edgar.getDocuments(tree, noOfDocuments=5)

        for doc in docs:
            #print(comp)
            year -= 1
            
            docs_df = docs_df.append({'Filing_Type': '10-K','Filing_Date': year,'Company':comp, 'Text' :doc,'Label' :'1'},ignore_index=True)
    return docs_df



In [2]:
# Includes the stock data
def ten_k_files():
    company_dict = {"FANG": "0001539838", "RSPP":"0001588216", "CXO":" 0001358071", "MDU": "0000067716",
               "LPI": "0001528129", "NFX": "0000912750", "ECA": "0001157806", "GPOR": "0000874499",
               "RRC": "0000315852", "EQT": "0000033213", "SM": "0000893538"}
    docs_df = pd.DataFrame(columns=['Filing_Type','Filing_Date','Company','Text','Label', 
                                    'Filings', 'date', '1_Day', '1_Month', '3_Month', 'close',
       'close_1_Day', 'close_1_Month', 'close_3_Month'])
    for comp in company_dict:
        print(comp)
        cik = company_dict[comp]
        company = edgar.Company(comp, cik)
        tree = company.getAllFilings(filingType = "10-K")
#        docs = edgar.getDocuments(tree, noOfDocuments=5)
        
        df_comp_dates = pd.DataFrame()
        
        file = f'../data/{comp}.csv'
        df_comp = pd.read_csv(file, encoding="ISO-8859-1", index_col=[0])
        df_comp.drop(['company', 'open', 'high', 'low','adjusted_close', 'volume', 'dividend_amount', 'split_coefficient' ,'crude_price'], axis=1, inplace=True)
        df_comp['date'] = pd.to_datetime(df_comp['date'])

        df_comp = df_comp.set_index('date')

        df_comp = df_comp.resample('D').mean()
        df_comp = df_comp.fillna(method='ffill')

        df_comp = df_comp.reset_index(drop=False)

        df_comp_dates = pd.read_excel("../data/dates/Document_Filing_Dates.xlsx", sheet_name=comp)
        df_comp_dates.rename(columns = {'Filing Date':'date'}, inplace = True)


        df_comp_dates['1_Day'] = df_comp_dates['date'] + np.timedelta64(1,'D')
        df_comp_dates['1_Month'] = df_comp_dates['date'] + np.timedelta64(30,'D')
        df_comp_dates['3_Month'] = df_comp_dates['date'] + np.timedelta64(90,'D')

        df_comp_dates = pd.merge(df_comp_dates, df_comp, on='date' ,how='left')

        df_comp.rename(columns = {'date':'1_Day'}, inplace = True)
        df_comp.rename(columns = {'close':'close_1_Day'}, inplace = True)

        df_comp_dates = pd.merge(df_comp_dates, df_comp, on='1_Day' ,how='left')

        df_comp.rename(columns = {'1_Day':'1_Month'}, inplace = True)
        df_comp.rename(columns = {'close_1_Day':'close_1_Month'}, inplace = True)

        df_comp_dates = pd.merge(df_comp_dates, df_comp, on='1_Month' ,how='left')

        df_comp.rename(columns = {'1_Month':'3_Month'}, inplace = True)
        df_comp.rename(columns = {'close_1_Month':'close_3_Month'}, inplace = True)

        df_comp_dates = pd.merge(df_comp_dates, df_comp, on='3_Month' ,how='left')
        
                    
        df_comp_dates[df_comp_dates['Filings'] == "10-K"]
                    
                    
        df_comp_dates.reset_index(drop=True)
        df_comp_dates = df_comp_dates.reset_index(drop=True).sort_values('date', ascending=False)
        
        comp_2018 = ['RSPP','NFX']
        comp_2019 = ['FANG','CXO','MDU','PE','MRO','LPI','ECA','GPOR','RRC','EQT',"SM"]
        count = 0
        if comp in comp_2018:
            df_comp_dates_2018 = df_comp_dates[(df_comp_dates['Filings'] == "10-K") & (df_comp_dates["date"] < np.datetime64('2019-01-01'))].reset_index()
            start_year=2018
            docs = edgar.getDocuments(tree, noOfDocuments=5)
            #print(len(df_comp_dates_2018))
            print("count 2018:" + str(len(docs)))
            
            for idx,doc in enumerate(docs):
                #print(idx)
                                
                fil = df_comp_dates_2018['Filings'][idx]
                day = df_comp_dates_2018['date'][idx]
                day_un = df_comp_dates_2018['1_Day'][idx]
                mnth = df_comp_dates_2018['1_Month'][idx]
                mnth_tre = df_comp_dates_2018['3_Month'][idx]
                close = df_comp_dates_2018['close'][idx]
                close_un = df_comp_dates_2018['close_1_Day'][idx]
                close_mnth_un = df_comp_dates_2018['close_1_Month'][idx]
                close_mnth_tre = df_comp_dates_2018['close_3_Month'][idx]
            
                    
                #print(fil,day,day_un,mnth,mtnh_tre,close,close_un,close_mnth_un,close_mnth_tre)
                docs_df = docs_df.append({'Filing_Type': fil,'Filing_Date': start_year,'Company':comp, 'Text' :doc,'Label' :'1', 
                                              'Filings':fil, 'date':day, '1_Day':day_un, '1_Month':mnth, '3_Month':mnth_tre, 'close':close,
                                              'close_1_Day':close_un, 'close_1_Month':close_mnth_un, 'close_3_Month':close_mnth_tre},ignore_index=True)
                start_year -= 1

        else:
            df_comp_dates = df_comp_dates[(df_comp_dates['Filings'] == "10-K") & (df_comp_dates["date"] < np.datetime64('2019-03-01'))].reset_index()
            year_start = 2019
            docs = edgar.getDocuments(tree, noOfDocuments=6)
            print(len(df_comp_dates))
            print("count 2018:" + str(len(docs)))
            
            for idx,doc in enumerate(docs):
                #print("two" + str(idx))
                #print("-----")
    
                fil = df_comp_dates['Filings'][idx]
                day = df_comp_dates['date'][idx]
                day_un = df_comp_dates['1_Day'][idx]
                mnth = df_comp_dates['1_Month'][idx]
                mnth_tre = df_comp_dates['3_Month'][idx]
                close = df_comp_dates['close'][idx]
                close_un = df_comp_dates['close_1_Day'][idx]
                close_mnth_un = df_comp_dates['close_1_Month'][idx]
                close_mnth_tre = df_comp_dates['close_3_Month'][idx]
            
                    
                #print(fil,day,day_un,mnth,mtnh_tre,close,close_un,close_mnth_un,close_mnth_tre)
                docs_df = docs_df.append({'Filing_Type': fil,'Filing_Date': year_start,'Company':comp, 'Text' :doc,'Label' :'1', 
                                              'Filings':fil, 'date':day, '1_Day':day_un, '1_Month':mnth, '3_Month':mnth_tre, 'close':close,
                                              'close_1_Day':close_un, 'close_1_Month':close_mnth_un, 'close_3_Month':close_mnth_tre},ignore_index=True)
                year_start -= 1
        
            
        #docs_df = pd.merge(docs_df, df_comp_dates, on='Filings' ,how='left')
        #df_comp_dates
    return docs_df

In [34]:
docs_df = ten_k_files()

FANG
6
count 2018:6
RSPP
count 2018:5
CXO
6
count 2018:6
MDU
6
count 2018:6
LPI
6
count 2018:6
NFX
count 2018:5
ECA
3
count 2018:3
GPOR
6
count 2018:6
RRC
6
count 2018:6
EQT
6
count 2018:6
SM
6
count 2018:6


In [35]:
docs_df['Filing_Date'] = docs_df['Filing_Date'].apply(str)

In [36]:
#full list of companies
docs_df['Index'] = docs_df[['Filing_Date', 'Company']].apply(lambda x: ''.join(x), axis=1)
index_df = docs_df['Index']
index_df.to_csv("index.csv")
#calculates stock deltas
docs_df['1_Day_Change'] = docs_df['close_1_Day']-docs_df['close']
docs_df['1_Month_Change'] = docs_df['close_1_Month']-docs_df['close']
docs_df['3_Month_Change'] = docs_df['close_3_Month']-docs_df['close']
#saves full data
full_docs_df = docs_df 
full_docs_df.head()

,Filing_Type,Filing_Date,Company,Text,Label,Filings,date,1_Day,1_Month,3_Month,close,close_1_Day,close_1_Month,close_3_Month,Index,1_Day_Change,1_Month_Change,3_Month_Change
0,10-K,2019,FANG,"\n10-K/A\n1\ndiamondback201810-ka.htm\nDIAMONDBACK 10-K/A\n\n\n\n\t\n\t\t\n\t\t\n\t\tDocument\n\t\n\t\n UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K/AAmendment No. 1 ýANNUAL REPORT UNDER SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2018 ORoTRANSITION REPORT UNDER SECTION 13 OR 15(d) OF SECURITIES EXCHANGE ACT OF 1934Commission File Number 001-35700 Diamondback Energy, Inc.(Exact Name of Registrant As Specified in Its Charter) Delaware 45-4502447(State or Other Jurisdiction ofIncorporation or Organization) (IRS EmployerIdentification Number) 500 West Texas, Suite 1200Midland, Texas 79701(Address of Principal Executive Offices) (Zip Code)(Registrant Telephone Number, Including Area Code): (432) 221-7400 Securities registered pursuant to Section 12(b) of the Act: Title of Each Class Name of Each Exchange on Which Registered Common Stock, par value $0.01 per share The N...",1,10-K,2019-02-25,2019-02-26,2019-03-27,2019-05-26,105.29,105.51,102.35,NaN,2019FANG,0.22,-2.94,NaN
1,10-K,2018,FANG,"\n10-K\n1\ndiamondback201810-k.htm\nDIAMONDBACK 10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\tDocument\n\t\n\t\nTable of Contents UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K ýANNUAL REPORT UNDER SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2018 OR¨TRANSITION REPORT UNDER SECTION 13 OR 15(d) OF SECURITIES EXCHANGE ACT OF 1934Commission File Number 001-35700 Diamondback Energy, Inc.(Exact Name of Registrant As Specified in Its Charter) Delaware 45-4502447(State or Other Jurisdiction ofIncorporation or Organization) (IRS EmployerIdentification Number) 500 West Texas, Suite 1200Midland, Texas 79701(Address of Principal Executive Offices) (Zip Code)(Registrant Telephone Number, Including Area Code): (432) 221-7400 Securities registered pursuant to Section 12(b) of the Act: Title of Each Class Name of Each Exchange on Which Registered Common Stock, par value $0.01 per share The Nasdaq...",1,10-K,2018-02-15,2018-02-16,2018-03-17,2018-05-16,125.15,123.88,125.51,132.13,2018FANG,-1.27,0.36,6.98
2,10-K,2017,FANG,"\n10-K\n1\ndiamondback201710-k.htm\nDIAMONDBACK 10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\tDocument\n\t\n\t\nTable of Contents UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K ýANNUAL REPORT UNDER SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2017 OR¨TRANSITION REPORT UNDER SECTION 13 OR 15(d) OF SECURITIES EXCHANGE ACT OF 1934Commission File Number 001-35700 Diamondback Energy, Inc.(Exact Name of Registrant As Specified in Its Charter) Delaware 45-4502447(State or Other Jurisdiction ofIncorporation or Organization) (IRS EmployerIdentification Number) 500 West Texas, Suite 1200Midland, Texas 79701(Address of Principal Executive Offices) (Zip Code)(Registrant Telephone Number, Including Area Code): (432) 221-7400 Securities registered pursuant to Section 12(b) of the Act: Title of Each Class Name of Each Exchange on Which Registered Common Stock, par value $0.01 per share The Nasdaq...",1,10-K,2017-02-15,2017-02-16,2017-03-17,2017-05-16,111.84,109.01,104.69,103.96,2017FANG,-2.83,-7.15,-7.88
3,10-K,2016,FANG,"\n10-K\n1\ndiamondback201610-k.htm\nDIAMONDBACK 10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\tDocument\n\t\n\t\nTable of Contents UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K ýANNUAL REPORT UNDER SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2016 OR¨TRANSITION REPORT UNDER SECTION 13 OR 15(d) OF SECURITIES EXCHANGE ACT OF 1934Commission File Number 001-35700 Diamondback Energy, Inc.(Exact Name of Registrant As Specified in Its Charter) Delaware 45-4502447(St

In [37]:
#Cleaning the dataframe and calculating the actual labels
docs_df = docs_df[['Index','Text','Filing_Type', 'Filing_Date', 'Company', 'Label', '1_Day_Change', '1_Month_Change', '3_Month_Change']]
docs_df['Label_1_Day'] = np.where(docs_df['1_Day_Change']>=0, '1', '0')
docs_df['Label_1_Month'] = np.where(docs_df['1_Month_Change']>=0, '1', '0')
docs_df['Label_3_Month'] = np.where(docs_df['3_Month_Change']>=0, '1', '0')


C:\Users\yural\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\yural\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\yural\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a co

In [38]:
#sets index
docs_df = docs_df.set_index('Index')

In [39]:
docs_df

,Text,Filing_Type,Filing_Date,Company,Label,1_Day_Change,1_Month_Change,3_Month_Change,Label_1_Day,Label_1_Month,Label_3_Month
Index,,,,,,,,,,,
2019FANG,"\n10-K/A\n1\ndiamondback201810-ka.htm\nDIAMONDBACK 10-K/A\n\n\n\n\t\n\t\t\n\t\t\n\t\tDocument\n\t\n\t\n UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K/AAmendment No. 1 ýANNUAL REPORT UNDER SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2018 ORoTRANSITION REPORT UNDER SECTION 13 OR 15(d) OF SECURITIES EXCHANGE ACT OF 1934Commission File Number 001-35700 Diamondback Energy, Inc.(Exact Name of Registrant As Specified in Its Charter) Delaware 45-4502447(State or Other Jurisdiction ofIncorporation or Organization) (IRS EmployerIdentification Number) 500 West Texas, Suite 1200Midland, Texas 79701(Address of Principal Executive Offices) (Zip Code)(Registrant Telephone Number, Including Area Code): (432) 221-7400 Securities registered pursuant to Section 12(b) of the Act: Title of Each Class Name of Each Exchange on Which Registered Common Stock, par value $0.01 per share The N...",10-K,2019,FANG,1,0.22,-2.94,NaN,1,0,0
2018FANG,"\n10-K\n1\ndiamondback201810-k.htm\nDIAMONDBACK 10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\tDocument\n\t\n\t\nTable of Contents UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K ýANNUAL REPORT UNDER SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2018 OR¨TRANSITION REPORT UNDER SECTION 13 OR 15(d) OF SECURITIES EXCHANGE ACT OF 1934Commission File Number 001-35700 Diamondback Energy, Inc.(Exact Name of Registrant As Specified in Its Charter) Delaware 45-4502447(State or Other Jurisdiction ofIncorporation or Organization) (IRS EmployerIdentification Number) 500 West Texas, Suite 1200Midland, Texas 79701(Address of Principal Executive Offices) (Zip Code)(Registrant Telephone Number, Including Area Code): (432) 221-7400 Securities registered pursuant to Section 12(b) of the Act: Title of Each Class Name of Each Exchange on Which Registered Common Stock, par value $0.01 per share The Nasdaq...",10-K,2018,FANG,1,-1.27,0.36,6.98,0,1,1
2017FANG,"\n10-K\n1\ndiamondback201710-k.htm\nDIAMONDBACK 10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\tDocument\n\t\n\t\nTable of Contents UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K ýANNUAL REPORT UNDER SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2017 OR¨TRANSITION REPORT UNDER SECTION 13 OR 15(d) OF SECURITIES EXCHANGE ACT OF 1934Commission File Number 001-35700 Diamondback Energy, Inc.(Exact Name of Registrant As Specified in Its Charter) Delaware 45-4502447(State or Other Jurisdiction ofIncorporation or Organization) (IRS EmployerIdentification Number) 500 West Texas, Suite 1200Midland, Texas 79701(Address of Principal Executive Offices) (Zip Code)(Registrant Telephone Number, Including Area Code): (432) 221-7400 Securities registered pursuant to Section 12(b) of the Act: Title of Each Class Name of Each Exchange on Which Registered Common Stock, par value $0.01 per share The Nasdaq...",10-K,2017,FANG,1,-2.83,-7.15,-7.88,0,0,0
2016FANG,"\n10-K\n1\ndiamondback201610-k.htm\nDIAMONDBACK 10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\tDocument\n\t\n\t\nTable of Contents UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K ýANNUAL REPORT UNDER SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2016 OR¨TRANSITION REPORT UNDER SECTION 13 OR 15(d) OF SECURITIES EXCHANGE ACT OF 1934Commission File Number 001-35700 Diamondback Energy, Inc.(Exact Name of Registrant As Specified in Its Charter) Delaware 45-4502447(State or Other Jurisdiction ofIncorporation or Organization) (IRS EmployerIdentification Number) 500 West Texas, Suite 1200Midland, Texas 79701(Address of Principal Executive Offices) (Zip Code)(Registrant Telephone Number, Including Area Code): (432) 221-

In [40]:
# Vectorization methods
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Classification model
from sklearn.linear_model import LinearRegression

### Train/Test Split

Train will be a 80% split of the corpus of documents comprising of 10K and 10Qs for the companies spanning time.
Test will be the 20% remaining documents.

Documents will be the features, and Labels will be the stock price change label (positive or negative)

There will be 3 main models: one for the stock change at 1 day, one for the stock change at 1 month, one for the stock change at 3 months (before the next quarter)

In [41]:
#Set the Label column to be Label_1_Month for the second model (1 month)
docs_df['Label'] = docs_df['Label_1_Month']
docs_df.head()

,Text,Filing_Type,Filing_Date,Company,Label,1_Day_Change,1_Month_Change,3_Month_Change,Label_1_Day,Label_1_Month,Label_3_Month
Index,,,,,,,,,,,
2019FANG,"\n10-K/A\n1\ndiamondback201810-ka.htm\nDIAMONDBACK 10-K/A\n\n\n\n\t\n\t\t\n\t\t\n\t\tDocument\n\t\n\t\n UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K/AAmendment No. 1 ýANNUAL REPORT UNDER SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2018 ORoTRANSITION REPORT UNDER SECTION 13 OR 15(d) OF SECURITIES EXCHANGE ACT OF 1934Commission File Number 001-35700 Diamondback Energy, Inc.(Exact Name of Registrant As Specified in Its Charter) Delaware 45-4502447(State or Other Jurisdiction ofIncorporation or Organization) (IRS EmployerIdentification Number) 500 West Texas, Suite 1200Midland, Texas 79701(Address of Principal Executive Offices) (Zip Code)(Registrant Telephone Number, Including Area Code): (432) 221-7400 Securities registered pursuant to Section 12(b) of the Act: Title of Each Class Name of Each Exchange on Which Registered Common Stock, par value $0.01 per share The N...",10-K,2019,FANG,0,0.22,-2.94,NaN,1,0,0
2018FANG,"\n10-K\n1\ndiamondback201810-k.htm\nDIAMONDBACK 10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\tDocument\n\t\n\t\nTable of Contents UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K ýANNUAL REPORT UNDER SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2018 OR¨TRANSITION REPORT UNDER SECTION 13 OR 15(d) OF SECURITIES EXCHANGE ACT OF 1934Commission File Number 001-35700 Diamondback Energy, Inc.(Exact Name of Registrant As Specified in Its Charter) Delaware 45-4502447(State or Other Jurisdiction ofIncorporation or Organization) (IRS EmployerIdentification Number) 500 West Texas, Suite 1200Midland, Texas 79701(Address of Principal Executive Offices) (Zip Code)(Registrant Telephone Number, Including Area Code): (432) 221-7400 Securities registered pursuant to Section 12(b) of the Act: Title of Each Class Name of Each Exchange on Which Registered Common Stock, par value $0.01 per share The Nasdaq...",10-K,2018,FANG,1,-1.27,0.36,6.98,0,1,1
2017FANG,"\n10-K\n1\ndiamondback201710-k.htm\nDIAMONDBACK 10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\tDocument\n\t\n\t\nTable of Contents UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K ýANNUAL REPORT UNDER SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2017 OR¨TRANSITION REPORT UNDER SECTION 13 OR 15(d) OF SECURITIES EXCHANGE ACT OF 1934Commission File Number 001-35700 Diamondback Energy, Inc.(Exact Name of Registrant As Specified in Its Charter) Delaware 45-4502447(State or Other Jurisdiction ofIncorporation or Organization) (IRS EmployerIdentification Number) 500 West Texas, Suite 1200Midland, Texas 79701(Address of Principal Executive Offices) (Zip Code)(Registrant Telephone Number, Including Area Code): (432) 221-7400 Securities registered pursuant to Section 12(b) of the Act: Title of Each Class Name of Each Exchange on Which Registered Common Stock, par value $0.01 per share The Nasdaq...",10-K,2017,FANG,0,-2.83,-7.15,-7.88,0,0,0
2016FANG,"\n10-K\n1\ndiamondback201610-k.htm\nDIAMONDBACK 10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\tDocument\n\t\n\t\nTable of Contents UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K ýANNUAL REPORT UNDER SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2016 OR¨TRANSITION REPORT UNDER SECTION 13 OR 15(d) OF SECURITIES EXCHANGE ACT OF 1934Commission File Number 001-35700 Diamondback Energy, Inc.(Exact Name of Registrant As Specified in Its Charter) Delaware 45-4502447(State or Other Jurisdiction ofIncorporation or Organization) (IRS EmployerIdentification Number) 500 West Texas, Suite 1200Midland, Texas 79701(Address of Principal Executive Offices) (Zip Code)(Registrant Telephone Number, Including Area Code): (432) 221-

In [42]:
### Assign X (data) and y (target)
X = docs_df.drop("Label", axis=1)
y = docs_df["Label"]
print(X.shape, y.shape)

(61, 10) (61,)


In [43]:
#train/test split at 80%
from sklearn.model_selection import train_test_split
XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2, random_state = 4)

In [44]:
#look at train/test split
XTrain
yTrain
XTest
yTest

Index
2015FANG    0
2016NFX     1
2015LPI     1
2014MDU     0
2018NFX     0
2017SM      0
2016EQT     1
2016CXO     1
2017LPI     0
2018LPI     1
2018RSPP    1
2016MDU     1
2017RSPP    1
Name: Label, dtype: object

In [45]:
#Create train/test dataframes
n_docs = 100000 #number of times to run model
train_docs = XTrain
train_labels = yTrain
test_docs = XTest
test_labels = yTest

train_df = pd.DataFrame({"text": train_docs['Text'], "labels": train_labels})
train_df

test_df = pd.DataFrame({"text": test_docs['Text'], "labels": test_labels})
test_df

print(f"Train Shape: {train_df.shape}")
print(f"Test Shape: {test_df.shape}")

Train Shape: (48, 2)
Test Shape: (13, 2)


In [46]:
train_df.head()

,text,labels
Index,,
2018CXO,"\n10-K\n1\ncxo201710K.htm\nFORM 10-K\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\nUNITED\nSTATES SECURITIES AND EXCHANGE COMMISSION \n\nWashington, D.C. 20549 \n\n \n\nFORM 10-K \n\n\n\n\n \n \n \n \n \n \n \n \n \n \n \n\n \n ☑ \n \n \n \n \n \n ANNUAL\n REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n \n \n \n\n\n\n\n\n\nFor the\nfiscal year ended December 31, 2017\n\nor \n\n\n\n\n \n \n \n \n \n \n \n \n \n \n \n\n \n o \n \n \n \n \n \n TRANSITION\n REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n \n \n \n \n\n\n\n\n\n\nFor the\ntransition period from \n \nto \n \n\n \n\nCommission\nfile number: 1-33615 \n\n \n\nConcho\nResources Inc. \n\n(Exact name of registrant\nas specified in its charter) \n\n\n \n \n \n \n \n \n \n \n \n \n \n\n \n Delaware\n \n \n \n \n ...",1
2016LPI,"\n10-K\n1\na2015form10-k.htm\n10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\t10-K\n\t\n\t\nUNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549FORM 10-Ký ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2015oro TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934Commission file number: 001-35380Laredo Petroleum, Inc.(Exact name of registrant as specified in its charter)Delaware(State or other jurisdiction ofincorporation or organization) 45-3007926(I.R.S. EmployerIdentification No.)15 W. Sixth Street, Suite 900Tulsa, Oklahoma(Address of principal executive offices) 74119(Zip code)(918) 513-4570(Registrant's telephone number, including area code)Securities Registered Pursuant to Section 12(b) of the Act:Title of Each Class Name of Each Exchange On Which RegisteredCommon Stock, $0.01 par value per share New York Stock ExchangeSecurities Registered Pursuant to Section 12(g) ...",1
2019CXO,"\n10-K\n1\ncxo201810K.htm\nFORM 10-K\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\nUNITED STATES SECURITIES AND EXCHANGE COMMISSION \n\nWashington,\nD.C. 20549 \n\n \n\nFORM 10-K \n\n\n\n\n \n \n \n \n \n \n \n \n \n \n \n\n \n ☑ \n \n \n \n \n \n ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES\n EXCHANGE ACT OF 1934 \n \n \n\n\n\n\n\n\nFor the fiscal year ended December 31, 2018\n\nor \n\n\n\n\n \n \n \n \n \n \n \n \n \n \n \n\n \n ☐ \n \n \n \n \n \n TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE\n SECURITIES EXCHANGE ACT OF 1934 \n \n \n \n\n\n\n\n\n\nFor the transition period from \n \nto \n \n\n \n\nCommission file number: 1-33615 \n\n \n\nConcho Resources Inc. \n\n(Exact name of registrant as specified in its charter) \n\n\n \n \n \n \n \n \n \n \n \n \n \n\n \n Delaware \n \n \n \n \n \n 76-...",0
2019EQT,"\n10-K/A\n1\na19-8685_110ka.htm\n10-K/A\n\n\n\n\n\n\n\n\n\n \n\n\nTable of Contents\n \nUNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\n_________________________________________________\n \nFORM 10-K/A\nAmendment No. 1 to Form 10-K\n_________________________________________________\n \n(Mark One)\n \nx ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n \nFOR THE FISCAL YEAR ENDED DECEMBER 31, 2018\n \no TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n \nor\n \nFOR THE TRANSITION PERIOD FROM: ______ TO: _______\n \nCOMMISSION FILE NUMBER 001-03551\n \nEQT CORPORATION\n(Exact name of registrant as specified in its charter)\n \n\n\n\n\nPENNSYLVANIA \n\n \n\n25-0464690 \n\n\n(State or other jurisdiction of incorporation or organization) \n\n \n\n(IRS Employer Identification No.) \n\n ...",1
2014CXO,"\n10-K\n1\nForm_10_K.htm\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n \n\n\n\n \n\n\n\nUNITED STATES SECURITIES AND EXCHANGE\nCOMMISSION \n\nWashington, D.C. 20549 \n\n \n\nFORM 10-K \n\n\n\n\n \n \n \n \n \n \n \n \n \n \n \n\n \n þ \n \n \n \n \n \n ANNUAL REPORT PURSUANT TO SECTION 13 OR\n 15(d) OF THE SECU

## CountVectorization

Turn the raw text from TrainText and TestText into feature vectors so that we can use them in our model.
We vectorize the text in 2 steps: 
1. First, we `fit`, the training data to our vectorizer to compute the vocabulary (feature set). 
2. Then, we `transform` with our text for both train and test to count the number occurrences for each word in our vocabulary.

The output of the CountVectorizer's `transform` task is a [sparse matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix), which condenses the matrix values to avoid storing an excessive amount of zeros.

In [47]:
vectorizer = CountVectorizer(stop_words='english') ##could limit the words being used in here
vectorizer.fit(train_df['text']) ##fit training data's text to get the feature set
train_vecs = vectorizer.transform(train_df['text']) ##transform with training to count word frequencies
test_vecs = vectorizer.transform(test_df['text']) ##transform with test data to count word frequencies

In [48]:
all_df = pd.DataFrame({"text": docs_df['Text']})

all_vecs = vectorizer.transform(all_df['text']) ##transform with all data to count word frequencies


In [49]:
#full vocabulary size for training data
print(f"Number of documents: {train_vecs.shape[0]}")
print(f"Size of vocabulary: {train_vecs.shape[1]}")

#full vocabulary size for testing data
print(f"Number of documents: {test_vecs.shape[0]}")
print(f"Size of vocabulary: {test_vecs.shape[1]}")

#full vocabulary size for all data
print(f"Number of documents: {all_vecs.shape[0]}")
print(f"Size of vocabulary: {all_vecs.shape[1]}") 

Number of documents: 48
Size of vocabulary: 31179
Number of documents: 13
Size of vocabulary: 31179
Number of documents: 61
Size of vocabulary: 31179


In [50]:
#get rid of non-zero values because it's a sparse matrix 
# Train
print(f"Number of TRAINING non-zero features: {train_vecs.nnz}")
print(f"Number of TRAINING zero features: {(train_vecs.shape[0]*train_vecs.shape[1])-train_vecs.nnz}")

# Test
print(f"Number of TEST non-zero features: {test_vecs.nnz}")
print(f"Number of TEST zero features: {(test_vecs.shape[0]*test_vecs.shape[1])-test_vecs.nnz}")

# All
print(f"Number of TEST non-zero features: {all_vecs.nnz}")
print(f"Number of TEST zero features: {(all_vecs.shape[0]*all_vecs.shape[1])-all_vecs.nnz}")

Number of TRAINING non-zero features: 212606
Number of TRAINING zero features: 1283986
Number of TEST non-zero features: 60587
Number of TEST zero features: 344740
Number of TEST non-zero features: 273193
Number of TEST zero features: 1628726


### Display all terms

Displays and exports all_vecs word frequencies to .csv

In [51]:

df_all_counts = pd.DataFrame(all_vecs.toarray(), 
                         columns=vectorizer.get_feature_names())[:100].T
df_all_counts.to_csv("all10kfinal2.csv")


## Term Frequency-Inverse Document Frequency (TF-IDF)

Tf-idf is a statistical representation of how relevant a word is to a particular document within a corpus. _Relevance_, in this scenario, can be defined as how much information a word provides about the context of one document vs all other documents in the corpus. 

In short, tf-idf is calculated by comparing the number of times that a particular terms occurs in a given document vs the number of other documents in the corpus that contain that word. A word that frequently occurs in 1 document, but only occurs in a very small number of other documents will have a high tf-idf score.

The calculation for tf-idf is the product of two smaller calculations:

$$TF_{i,j} = \frac{Number~of~times~word_{i}~occurs~in~document_{j}}{Total~number~of~words~in~document_{j}}$$


$$IDF_{i} = log(\frac{Total~number~of~documents~in~corpus}{Number~of~documents~that~contain~word_{i}})$$

This might actually be useful for us because we can see if specific terms are especially important to certain documents relative to others -- this only matters because maybe we can see when acquisitions might be on the horizon, for example.

In [52]:
#tf-idf vectorization using calculation
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_vectorizer.fit(train_df['text'])
train_tfidf_vecs = tfidf_vectorizer.transform(train_df['text'])
test_tfidf_vecs = tfidf_vectorizer.transform(test_df['text'])

### Display a few terms and their tf-idf scores for a few documents

This is only meant to be used for demonstration purposes. The cell below has no impact on the actual execution of our task. Also, this cell is only intended for use when the number of documents is small (<100), otherwise it will likely only display a bunch of zeros.

In [85]:
#alice note -- obviously need to clean this up a bit.
#df_tfidf = pd.DataFrame(train_tfidf_vecs.toarray(), 
                         #columns=tfidf_vectorizer.get_feature_names())[:15].T
#df_tfidf.tail(20)

#### Comparison of the representation of different words vs. straight up word frequency

In [51]:
#pd.DataFrame({"TF-IDF of stock":df_tfidf.loc['stock'], "CountVectorizer: Stock":df_counts.loc['stock']})

In [ ]:
#pd.DataFrame({"TF-IDF of drilling":df_tfidf.loc['drilling'], "CountVectorizer: Drilling":df_counts.loc['drilling']})

# LDA Modeling

uses bag of words model (after term-document matrix has already been created) - used as dimension reduction or noise reducing.

need to figure out optimum number of topics

In [53]:
import os.path
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

In [54]:
all_10k_dict = vectorizer.get_feature_names()


In [55]:
#fit feature models to LDA Model
from sklearn.decomposition import LatentDirichletAllocation

no_topics = 5

lda = LatentDirichletAllocation(n_components=no_topics, random_state=4, evaluate_every=1).fit(all_vecs)

C:\Users\yural\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


In [56]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic: {topic_idx}")
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
display_topics(lda, all_10k_dict, no_top_words)

Topic: 0
gas company december 31 natural oil production net cash 2014
Topic: 1
gas company oil december 31 natural million financial production net
Topic: 2
company million gas 31 december natural 10 operations assets tax
Topic: 3
31 oil gas company natural december million tagged financial ended
Topic: 4
gas company december oil natural 31 million 2016 financial 2015


In [57]:
pyLDAvis.sklearn.prepare(lda, all_vecs, vectorizer)


C:\Users\yural\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.064668  0.044447       1        1  93.152739
2      0.056268 -0.051508       2        1   6.734310
3     -0.049907  0.007733       3        1   0.111960
4     -0.013105  0.008278       4        1   0.000496
0     -0.057923 -0.008949       5        1   0.000494, topic_info=      Category          Freq        Term         Total  loglift  logprob
term                                                                    
21432  Default  27520.000000         gas  27520.000000  30.0000  30.0000
24852  Default  23336.000000         oil  23336.000000  29.0000  29.0000
17591  Default  26398.000000     company  26398.000000  28.0000  28.0000
18998  Default  22333.000000    december  22333.000000  27.0000  27.0000
5979   Default  22128.000000          31  22128.000000  26.0000  26.0000
24344  Default  20770.000000     natural  20770.000000  25.0000  25.0000
26247  Default  14247.000000  production  14247.000000  24.0000  24.0000
24078  Default  17592.000000     million  17592.000000  23.0000  23.0000
21081  Default  15454.000000   financial  15454.000000  22.0000  22.0000
24392  Default  14171.000000         net  14171.000000  21.0000  21.0000
3915   Default  12024.000000        2016  12024.000000  20.0000  20.0000
3618   Default  11542.000000        2014  11542.000000  19.0000  19.0000
17090  Default  12479.000000        cash  12479.000000  18.0000  18.0000
3760   Default  11562.000000        2015  11562.000000  17.0000  17.0000
21403  Default  10328.000000      future  10328.000000  16.0000  16.0000
27364  Default  11099.000000    reserves  11099.000000  15.0000  15.0000
24937  Default  12304.000000  operations  12304.000000  14.0000  14.0000
18498  Default  11530.000000       costs  11530.000000  13.0000  13.0000
1275   Default  11778.000000          10  11778.000000  12.0000  12.0000
20126  Default   9837.000000       ended   9837.000000  11.0000  11.0000
26387  Default   8331.000000  properties   8331.000000  10.0000  10.0000
26080  Default   8748.000000       price   8748.000000   9.0000   9.0000
30555  Default   9987.000000       value   9987.000000   8.0000   8.0000
16057  Default   9632.000000      assets   9632.000000   7.0000   7.0000
31046  Default  10310.000000        year  10310.000000   6.0000   6.0000
29097  Default   9225.000000       stock   9225.000000   5.0000   5.0000
19864  Default   7677.000000    drilling   7677.000000   4.0000   4.0000
22207  Default   9415.000000      income   9415.000000   3.0000   3.0000
26099  Default   7972.000000      prices   7972.000000   2.0000   2.0000
4069   Default   9030.000000        2017   9030.000000   1.0000   1.0000
...        ...           ...         ...           ...      ...      ...
17591   Topic5      0.064894     company  26398.215144  -0.6985  -5.2505
5979    Topic5      0.055520          31  22128.299377  -0.6781  -5.4065
3618    Topic5      0.037985        2014  11542.397905  -0.4068  -5.7861
17090   Topic5      0.039321        cash  12479.200912  -0.4503  -5.7515
24392   Topic5      0.041737         net  14171.450812  -0.5178  -5.6918
24344   Topic5      0.051440     natural  20770.958263  -0.6911  -5.4828
21403   Topic5      0.033714      future  10328.733764  -0.4150  -5.9053
16057   Topic5      0.031812      assets   9632.332503  -0.4033  -5.9634
24852   Topic5      0.051212         oil  23336.232056  -0.8120  -5.4873
27364   Topic5      0.032684    reserves  11099.693536  -0.5180  -5.9363
24937   Topic5      0.033454  operations  12304.782102  -0.5978  -5.9131
18498   Topic5      0.031075       costs  11530.040216  -0.6065  -5.9868
21081   Topic5      0.035750   financial  15454.419479  -0.7593  -5.8467
3507    Topic5      0.024466        2013   7830.105659  -0.4587  -6.2260
24078   Topic5      0.036442     million  17592.631115  -0.8697  -5.8275
3760    Topic5      0.028586        2015  11562.153006  -0.6928  -6.0703
16

In [59]:
lda_output = lda.transform(all_vecs)

# column names
topicnames = ["Topic" + str(i) for i in range(no_topics)]

# index names
docnames = ["Doc" + str(i) for i in range(len(all_df))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,0,1,0,0,0,1
Doc1,0,1,0,0,0,1
Doc2,0,1,0,0,0,1
Doc3,0,1,0,0,0,1
Doc4,0,1,0,0,0,1
Doc5,0,1,0,0,0,1
Doc6,0,1,0,0,0,1
Doc7,0,1,0,0,0,1
Doc8,0,1,0,0,0,1
Doc9,0,1,0,0,0,1


# Modeling -- classifying whether certain terms appearing will result in a positive or negative stock price

Vectorizing our data has converted our text data into a numeric feature set. Using these vectors, we can now begin to develop machine learning models for things like classification.

To further this model, look into better preprocessing, regression regularization, vocabulary pruning for feature selection, and hyperparameter tuning.

In [60]:
#determine regression model
from sklearn.linear_model import LinearRegression
classifier = LinearRegression()
classifier

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [61]:
classifier.fit(train_vecs, train_df['labels'])
predictions = classifier.predict(test_vecs)
print(f"Prediction   {predictions[:20]}")
print(f"Actual labels: {test_df['labels']}")

Prediction   [ 1.27711728  1.55119639  0.26674799  1.54678603 -0.25101494 -0.39944807
 -0.95541386  1.1109473  -0.72290399 -0.0233111   1.80438688  1.61649944
  0.74528565]
Actual labels: Index
2015FANG    0
2016NFX     1
2015LPI     1
2014MDU     0
2018NFX     0
2017SM      0
2016EQT     1
2016CXO     1
2017LPI     0
2018LPI     1
2018RSPP    1
2016MDU     1
2017RSPP    1
Name: labels, dtype: object


In [62]:
#Run a linear regression classification on the count vectors
count_linearReg = LinearRegression()
count_linearReg.fit(train_vecs, train_df['labels']) 
#train_vecs = frequency/ terms (unique features for this document)
#['labels'] is labeling if the change in stock price was positive or negative for the time frame
count_preds = count_linearReg.predict(test_vecs) 
#model.coef_ -> gives you coefficient values for all of terms. then look to see which have high weights / low weights. 
#terms are being weighted off of the change in the stock price. 
#if you see term in high weight (occurrence results in stock going up), (if low / negative weight, stock going down)

# Calculate the percentage of accurate predictions
accuracy = np.mean(count_preds==test_df['labels'])
print(f"LinearReg CountVectorizer accuracy: {accuracy}") ##linear regression, fit with the frequency*weights, 

LinearReg CountVectorizer accuracy: 0.0


### Run a logistic regression classification on the TF-IDF vectors

In [63]:
tfidf_linReg = LinearRegression()
tfidf_linReg.fit(train_tfidf_vecs, train_df['labels'])
tfidf_preds = tfidf_linReg.predict(test_tfidf_vecs)

# Calculate the percentage of accurate predictions
accuracy = np.mean(tfidf_preds==test_df['labels'])
print(f"LinReg TF-IDF accuracy: {accuracy}")

LinReg TF-IDF accuracy: 0.0


### View the terms with the highest coefficient values for each category

Notice that the terms highly weighted for each category seem to have highly negative weights for other categories. If we were to use more similarly related categories, we may not see such drastic differences.

Ignore the code behind this table. It is poorly written, but demonstrates the correct results.

In [64]:
from utils import getTopCoefs

#getTopCoefs(num_terms=5, model=tfidf_linReg, class_labels=train_df['labels'], feature_names=tfidf_vectorizer.get_feature_names())

ImportError: cannot import name 'getTopCoefs' from 'utils' (C:\Users\yural\Desktop\OilAndGas\OilGasEarningsAnalysis\Edgar Python\utils.py)

### View coefficient weights for CountVectorizer features

In [65]:
#getTopCoefs(num_terms=5, model=count_linearReg, class_labels=train_df['labels'], feature_names=vectorizer.get_feature_names())

In [66]:
#Predicted inaccurately:
# Expand the max width of how our dataFrames display on screen
pd.options.display.max_colwidth = 1000

# Compile a dataframe with our text, the actual label, and the predicted label
final_df = pd.DataFrame({"text": test_df['text'], "Actual": test_df['labels'], "Prediction": tfidf_preds})

# Display the rows of our dataframe where the actual label and predicted label don't match
final_df.loc[(final_df['Actual'] != final_df['Prediction'])]

,text,Actual,Prediction
Index,,,
2015FANG,"\n10-K\n1\ndiamondback201510-k.htm\nDIAMONDBACK 10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\t10-K\n\t\n\t\nTable of Contents UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 FORM 10-K ýANNUAL REPORT UNDER SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2015 OR¨TRANSITION REPORT UNDER SECTION 13 OR 15(d) OF SECURITIES EXCHANGE ACT OF 1934Commission File Number 001-35700 Diamondback Energy, Inc.(Exact Name of Registrant As Specified in Its Charter) Delaware 45-4502447(State or Other Jurisdiction ofIncorporation or Organization) (IRS EmployerIdentification Number) 500 West Texas, Suite 1200Midland, Texas 79701(Address of Principal Executive Offices) (Zip Code)(Registrant Telephone Number, Including Area Code): (432) 221-7400 Securities registered pursuant to Section 12(b) of the Act: Title of Each Class Name of Each Exchange on Which Registered Common Stock, par value $0.01 per share The NASDAQ Sto...",0,1.502522
2016NFX,"\n10-K\n1\nnfx201510-k.htm\n10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\t10-K\n\t\n\t\n UNITED STATES SECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549Form 10-K þANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the fiscal year ended December 31, 2015or ¨TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the transition period from to .Commission file number: 1-12534Newfield Exploration Company(Exact name of registrant as specified in its charter)Delaware 72-1133047(State of incorporation) (I.R.S. Employer Identification No.)4 Waterway Square Place,Suite 100,The Woodlands, Texas 77380(Zip Code)(Address of principal executive offices) Registrant’s telephone number, including area code:(281) 210-5100Securities Registered Pursuant to Section 12(b) of the Act:Title of Each Class Name of Each Exchange on Which RegisteredCommon Stock, par value $0.01 per share New York Stock Exchan...",1,1.295990
2015LPI,"\n10-K\n1\na2014form10-k.htm\n10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\t2014 Form 10-K\n\t\n\t\nUNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549FORM 10-Ký ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended December 31, 2014oro TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934Commission file number: 001-35380Laredo Petroleum, Inc.(Exact name of registrant as specified in its charter)Delaware(State or other jurisdiction ofincorporation or organization) 45-3007926(I.R.S. EmployerIdentification No.)15 W. Sixth Street, Suite 900Tulsa, Oklahoma(Address of principal executive offices) 74119(Zip code)(918) 513-4570(Registrant's telephone number, including area code)Securities Registered Pursuant to Section 12(b) of the Act:Title of Each Class Name of Each Exchange On Which RegisteredCommon Stock, $0.01 par value per share New York Stock ExchangeSecurities Registered Pursuant to Sect...",1,0.276117
2014MDU,"\n10-K\n1\nmdu-12312013x10k.htm\n10-K\n\n\n\n\t\n\t\t\n\t\t\n\t\tWdesk | MDU-12.31.2013-10K\n\t\n\t\nUNITED STATES SECURITIES AND EXCHANGE COMMISSIONWASHINGTON, D.C. 20549FORM 10-K ýANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the fiscal year ended December 31, 2013 OR oTRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the transition period from _____________ to ______________Commission file number 1-3480MDU Resources Group, Inc.(Exact name of registrant as specified in its charter)Delaware 41-0423660(State or other jurisdiction of incorporation or organization) (I.R.S. Employer Identification No.)1200 West Century AvenueP.O. Box 5650Bismarck, North Dakota 58506-5650(Address of principal executive offices)(Zip Code)(701) 530-1000(Registrant's telephone number, including area code)Securities registered pursuant to Section 12(b) of the Act:Title of each class Na